# DiploDatos 2021


### Categorización de publicaciones de productos realizadas en Mercado Libre

### 04 - Aprendizaje Supervisado

#### Condiciones generales que aplican a todos los prácticos:

- Las notebooks tienen que ser 100% reproducibles, es decir al ejecutar las celdas tal cuál como se entrega la notebook se deben obtener los mismos resultados sin errores.
- Código legible, haciendo buen uso de las celdas de la notebook y en lo posible seguir estándares de código para *Python* (https://www.python.org/dev/peps/pep-0008/).
- Utilizar celdas tipo *Markdown* para ir guiando el análisis.
- Limpiar el output de las celdas antes de entregar el notebook (ir a `Kernel` --> `Restart Kernel and Clear All Ouputs`).
- Incluir conclusiones del análisis que se hizo en la sección "Conclusiones". Tratar de aportar valor en esta sección, ser creativo.

## 1. Consignas

El objetivo de este **TP** es iterar sobre el modelado, experimentando con distintas técnicas, y utilizar el *pipeline* de datos que se realizó en el **TP** pasado.

#### Sección 1: Modelado

- Implementar redes del tipo *feed foward* y *LSTM*.

#### Sección 2: Regularización

El objetivo de este bloque es experimentar con distintos métodos de regularización.
A continuación se deja una lista de distinas técnicas.
Investigar de qué se trata cada una e implementarlas.
Tener en cuenta que no es necesario implementar todo lo que se propone.

- Experimentar con distintos tamaños de modelo. Sacar / agregar capas como así también neuronas.

- Utilizar distinos valores de *batch size*.

- Experimentar utilizando *dropout* y *batch normalization*. (**HECHO**)

- Experimentar con distintos valores para el *learning rate*.

- Experimentar con técnicas de *weight decay* y *early stopping*.

#### Sección 3: Ajuste de Hiperparámetros

Este bloque es **opcional**, ya que puede ser complejo hacer la implementación en redes neuronales.

- Investigar, y en lo posible implemementar, alguna técnica de búsqueda de hiperparámetros como *Grid Search* y *Random Search*.

#### Sección 4: Documentación de Resultados

- Dejar documentado de algún lado (puede ser en un documento de texto aparte) los resultados de toda la experimentación de este **TP**. Es importante que queden claros los resultados de las métricas de cada modelo, cual de todos los modelos entrenados fue el superador, y una breve descripción del mismo.

## 2. Código y análisis

Instalaciones necesarias.

In [ ]:
!pip install Unidecode

Importaciones necesarias

In [ ]:
import pandas as pd
import numpy as np

Lectura de dataset reducido

In [ ]:
df_dataset = pd.read_csv('DataSet/dataset.csv')

Estudiamos el dataset brevemente antes de comenzar a operar sobre el mismo.

In [ ]:
df_dataset.describe()

In [ ]:
classes = np.sort(df_dataset.category.unique())

print(f'Dimensiones: {df_dataset.shape}')
print('----------')
print(f'Variables: {list(df_dataset.columns)}')
print('----------')
print(f'Categorías: {list(classes)}')

**Sección 0:**
Preparando el conjunto de datos.

In [ ]:
from scripts.utils import cleaning

df_dataset['clean_title'] = df_dataset.title.apply(cleaning)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_dataset, train_size=0.8, test_size=0.2, random_state=123)

print(f'Dimensiones Entrenamiento: {df_train.shape}')
print(f'Dimensiones Evaluación: {df_test.shape}')

In [ ]:
# Datos de Entrenamiento
X_train = df_train.clean_title.values
y_train = df_train.category.values
# Datos de Test
X_test = df_test.clean_title.values
y_test = df_test.category.values

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

word_tokenizer = Tokenizer()
# Aprendemos el Tokenizer en base a los datos de entrenamiento.
word_tokenizer.fit_on_texts(X_train)
# Longitud del Vocabulario en base a los datos de entrenamiento.
vocab_length = len(word_tokenizer.word_index) + 1

embedded_sentences_train = word_tokenizer.texts_to_sequences(X_train)
embedded_sentences_test = word_tokenizer.texts_to_sequences(X_test)

padded_sentences_train = pad_sequences(embedded_sentences_train, padding='post')
# Longitud de Sentencias en base a los datos de entrenamiento.
ammount_sentences, sentences_length = padded_sentences_train.shape

padded_sentences_test = pad_sequences(embedded_sentences_test, padding='post', maxlen=sentences_length)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(classes)

encoded_labels_train = le.transform(y_train)
encoded_labels_test = le.transform(y_test)

In [ ]:
from keras.layers.embeddings import Embedding

dense_vector_size = 30
embedding_layer = Embedding(vocab_length, dense_vector_size, input_length=sentences_length)

**Observación**

Solo trabajaremos con *custom embeddings* por dificultades encontradas al adaptar el modelo a los *word vectors* preentrenados de **fastText**.
Una tarea que quedará pendiente, será replantear el uso de esta herramienta.

**Sección 1**

In [ ]:
from scripts.models import ...

# TODO: Definir que modelos queremos utilizar...

models = {m1.name: m1.model, m2.name: m2.model, m3.name: m3.model}
weights = {m1.name: m1.filepath, m2.name: m2.filepath, m3.name: m3.filepath}

**TODO:**
Presentar los modelos utilizados.

**Sección 2**

En base a la lista de técnicas que tenemos disponible, realizamos una breve investigación de cada una antes de comenzar con su implementación.
A continuación mencionamos algunas de sus características relevantes.

**Experimentar con los tamaños del modelo**

La idea es modificar la arquitectura de la red.
Nuestro modelo `Baseline`, por dar un ejemplo, tiene **2** capas ocultas densas, con **256** y **128** neuronas respectivamente.
Es una arquitectura definida totalmente de manera arbitraria, y sin ningún fundamento para respaldarla.
Durante la experimentación podríamos modificar la cantidad de capas ocultas y/o la cantidad de neuronas en cada una.
Quizás reduciendo alguna de estas cantidades podríamos regularizar el aprendizaje y evitar el *overfitting* observado en el **TP** anterior.

**Distinos valores de *batch size***

El *batch size* determina la cantidad de muestras que serán propagadas por la red (en una iteración de *forward pass* y *backward pass*).
Utilizar un tamaño **pequeño** reduce el uso de memoria, y generalmente acelera el entrenamiento al actualizar los pesos luego de cada propagación.
Utilizar un tamaño **grande** mejora la precisión de la estimación del gradiente, al observar una mayor cantidad de datos de entrenamiento se puede realizar una actualización más certera de los pesos de la red.

**Utilizar *dropout* y *batch normalization***

**Dropout** es una técnica donde ciertas neuronas seleccionadas aleatoriamente son ignoradas durante el entrenamiento.
Lo cual significa que no contribuirán en la activación de neuronas durante el *forward pass*, y no se le aplicará ninguna actualización de pesos durante el *backward pass*.
El objetivo es que la red sea menos sensible a los pesos específicos de las neuronas, y que sea capaz de generalizar mejor.

**Batch Normalization** es una capa para normalizar la muestra que recibe.
Aplica una transformación que mantiene la *media* cercana a **0**, y su *desviación estándar* cercana a **1**.
El objetivo es evitar que ciertas características de los datos dominen el entrenamiento de la red neuronal por sobre las demás.

**Distintos valores de *learning rate***

Una red neuronal es entrenada utilizando el algoritmo de optimización **Descenso de Gradiente Estocástico** (*SGD*).
El *learning rate* es el hiperparámetro que controla cuanto cambiará el modelo en respuesta al error estimado cada vez que los pesos de la red son actualizados.
Una tasa de aprendizaje **pequeña** podría resultar en un entrenamiento largo, el cual puede no converger.
Una tasa de aprendizaje **grande** podría aprender un conjunto de pesos subóptimos, con un entrenamiento inestable.

**Utilizar *weight decay* y *early stopping***

**Weight Decay** es una técnica de regularización utilizada en redes neuronales.
La idea consiste en agregar un término a la pérdida, el cual sea proporcional a la magnitud de los pesos de la red.
De esta manera, durante el proceso de entrenamiento, se intentaría decrementar el valor de los pesos.

**Early Stopping** es un método que permite detener el entrenamiento una vez que la red neuronal deja de mejorar su rendimiento.
El problema que se intenta solucionar es la elección del número de épocas de entrenamiento.
Una cantidad de *epochs* demasiado **grande** puede producir *overfitting*.
Una cantidad de *epochs* demasiado **pequeña** puede producir *underfitting*.

In [ ]:
from scripts.train import ...

# TODO: Quizás sea necesario un entrenamiento más refinado...

training = {}
for name, model in models.items():
  print(f'{name}:')
  history = train(name, model, padded_sentences_train, encoded_labels_train)
  training[name] = history

In [ ]:
from scripts.metrics import plot_accuracy_loss

for name, train in training.items():
  plot_accuracy_loss(train, name)

In [ ]:
from scripts.metrics import show_balanced_accuracy

for name, model in models.items():
  show_balanced_accuracy(name, model, padded_sentences_test, encoded_labels_test, df_test)

In [ ]:
data = []
for name, model in models.items():
  loss, accuracy = model.evaluate(padded_sentences_test, encoded_labels_test, verbose=0)
  data.append({'Model Name': name, 'Test Loss': loss, 'Test Accuracy': accuracy})

df = pd.DataFrame(data)

df

In [ ]:
df.to_csv('Checkpoint/model_results.csv', header=True, index=False)

In [ ]:
chosen_model = # TODO: Elegir el modelo según los resultados observados.

In [ ]:
from sklearn.metrics import classification_report

probabilities = chosen_model.predict(padded_sentences_test, verbose=0)
predictions = np.argmax(probabilities, axis=-1)

print(classification_report(encoded_labels_test, predictions, target_names=classes))

In [ ]:
test_title = df_test.title
test_prediction = le.inverse_transform(predictions)

submission = pd.DataFrame(list(zip(test_title, test_prediction)), columns=['title', 'category'])
submission.to_csv('DataSet/dataset_submission.csv', header=True, index=False)

**Sección 3**

**Sección 4**

In [ ]:
# TO DO

## 3. Conclusiones

In [ ]:
# TO DO